# NVIDIA Meeting Recorder Agent — Pro & Refactored

This notebook demonstrates how to build an autonomous AI agent using xpander.ai, a Backend-as-a-Service platform for AI agents, powered by an NVIDIA NIM model.
The agent functions as a Meeting Recording Assistant, showcasing a low-abstraction approach to agent design with persistent memory,tool orchestration, and state management.  

Specifically, the AI Agent has these capabilities: 

* Access external APIs (Calendar, Meeting recorder service)
* Execute complex multi-turn operations (Recording setup, post-meeting asset retrieval)
* Maintain state across interactions
* Process and synthesize information from various sources (Calendar, Meeting details...)

## Prerequisites

Before coding, make sure you have the following:

- **Python** ≥ 3.10 installed on your system
- **Node.js** (for the xpander CLI)
- **LLM**: Access to an **NVIDIA NIM** model, specifically: llama-3.1-nemotron-ultra-253b-v1. You can get access to NVIDIA-hosted inference of this model by creating an account and generating an API key on [NVIDIA NIM API](https://build.nvidia.com/)
- Access to the xpander.ai Platform at https://app.xpander.ai
- .env file containing

    * XPANDER_API_KEY=...   # Get from https://app.xpander.ai later in the tutorial
    * NVIDIA_NIM_API_KEY=... # Get from https://build.nvidia.com
    * XPANDER_AGENT_ID=... # Get from https://app.xpander.ai later in the tutorial

## Installation

First, let's install the required packages:

In [ ]:
%pip install -r requirements.txt

# Set up and build the AI Agent
#
#
#### 1. Login to xpander.ai: 
#
##### Install xpander CLI on your terminal (skip if already installed)
```bash
npm install -g xpander-cli
xpander login    # Register or login to the xpander.ai platform. Once completed, copy XPANDER_API_KEY from the CLI response to your .env file
```
#### 2. Build the Agent:

Use the pre‑built xpander template below to create the agent with all necessary tools wired up.
#
<div style="text-align: center;">
  <a href="https://app.xpander.ai/templates/48039a71-c99c-4691-8b66-a6faca3ccbe4" target="_blank">
    <button style="background-color:#6741d9;color:white;padding:10px 20px;font-size:14px;border:none;border-radius:8px;cursor:pointer;">
      Build NVIDIA Meeting Recorder Agent with xpander Template
    </button>
  </a>
</div>

#
After completing the template import, make sure your agent looks similar to this in the user-interface, and that you complete the connection to Google Calendar (make sure there's no red exclamation mark on your Google Calendar tool)

<img src="/images/agent-screenshot.png" width="900">

<div style="
  border-left: 4px solid #42b983;
  padding-left: 1em;
  margin: 1em 0;
  font-size: 14px;
">
  <strong>💡 </strong> Quick rundown of the xpander.ai Agent Workbench:

  * The top nodes are source nodes - you use them to configure different ways to trigger your agent.

  * The box marked "Meeting Recording Agent - (Imported)" is a visual representation of your agent.

  * Nodes inside the agent are tools (for function calling). Every time the agent is triggered, these tools are presented to the model as its available tools in the inference call.
  
  * You can click the Settings (cog) button to see all the settings for your agent, such as instructions, memory settings, conversation starters, and more.

</div>

#
#
Copy the Agent ID from the URL (https://app.xpander.ai/agents/{agent-id}) and add it to your .env file as XPANDER_AGENT_ID={agent-id}.

For example: "e0ee265f-5f79-4aa3-34b5-5226623223e6f"

Next step: load the environment variables.

In [ ]:
import os, time, datetime as dt
from dotenv import load_dotenv
from xpander_sdk import XpanderClient, LLMProvider, Agent, Tokens, LLMTokens
from openai import OpenAI
import json

load_dotenv()

REQUIRED = ["XPANDER_API_KEY", "NVIDIA_NIM_API_KEY", "XPANDER_AGENT_ID"]
missing = [k for k in REQUIRED if os.getenv(k) is None]
if missing:
    raise EnvironmentError(f"Missing env vars: {', '.join(missing)}")

XPANDER_API_KEY    = os.getenv("XPANDER_API_KEY")
NVIDIA_NIM_API_KEY = os.getenv("NVIDIA_NIM_API_KEY")
AGENT_ID           = os.getenv("XPANDER_AGENT_ID")

print("✅ Environment ready — open your agent at",
      f"https://app.xpander.ai/agents/{AGENT_ID}")

#### 3. MeetingAgent Class - The core agent implementation

 The agent executes a multi-step processing loop that:
- Processes user input
- Manages context (including time)
- Generates LLM responses
- Extracts and executes tool calls
- Updates state and returns results

In [ ]:

class MeetingAgent:
    """Wrapper around an xpander agent backed by NVIDIA NIM."""

    MODEL = "nvidia/llama-3.1-nemotron-ultra-253b-v1"

    def __init__(self, xpander_key: str, nim_key: str, agent_id: str):
        self.client      = XpanderClient(api_key=xpander_key)
        self.agent: Agent= self.client.agents.get(agent_id=agent_id)
        self.agent.select_llm_provider(llm_provider=LLMProvider.NVIDIA_NIM)
        self.llm         = OpenAI(base_url="https://integrate.api.nvidia.com/v1",
                                  api_key=nim_key)

    # ------------------------------------------------------------------
    def run(self, prompt: str, *, thread_id: str | None = None):
        """Run a single prompt; returns (text, thread_id)."""
        self.agent.add_task(input=prompt, thread_id=thread_id)
        
        current_time = dt.datetime.now().isoformat(sep=' ', timespec='seconds')
        
        # add current time for fresh threads
        if not thread_id:
            self.agent.add_messages([{
                "role": "system",
                "content": f"Current time: {current_time}"
            }])

        usage = Tokens(worker=LLMTokens())   # token accounting
        t_start = time.perf_counter()

        while not self.agent.is_finished():
            t_loop = time.perf_counter()
            res = self.llm.chat.completions.create(
                model       = self.MODEL,
                messages    = self.agent.messages,
                temperature = 0,
                tools       = self.agent.get_tools(),
                tool_choice = self.agent.tool_choice,
            )
            
            # count tokens
            u = res.usage
            usage.worker.completion_tokens += u.completion_tokens
            usage.worker.prompt_tokens     += u.prompt_tokens
            usage.worker.total_tokens      += u.total_tokens

            # report execution llm metrics
            self.agent.report_llm_usage(
                res.model_dump(),
                time.perf_counter() - t_loop
            )
            
            # add llm response to the agent's memory
            self.agent.add_messages(res.model_dump())

            # extract tool calls and run (cloud)
            tool_calls = self.agent.extract_tool_calls(res.model_dump())
            self.agent.run_tools(tool_calls)

        result = self.agent.retrieve_execution_result()
        self.agent.report_execution_metrics(
            llm_tokens=usage,
            ai_model=self.MODEL
        )
        duration = time.perf_counter() - t_start
        print(f"⏱ Finished in {duration:.1f}s — status: {result.status}")
        return result.result, result.memory_thread_id


#### 4. Initialize agent & provide instructions:

In [ ]:

meeting_agent = MeetingAgent(XPANDER_API_KEY, NVIDIA_NIM_API_KEY, AGENT_ID)

TOOLS_JSON = json.dumps(meeting_agent.agent.get_tools())

# set the agent's instructions
meeting_agent.agent.instructions.general = f"""
Reasoning mode: ON
You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
To list user's calendar, use CalendarEventManagementGetCalendarEventsById
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of <TOOLCALL>[func_name1{{'params_name1': 'params_value1', 'params_name2': 'params_value2'...}}]</TOOLCALL>

You SHOULD NOT include any other text in the response.
Here is a list of functions in JSON format that you can invoke. 

<AVAILABLE_TOOLS>{TOOLS_JSON}</AVAILABLE_TOOLS>
"""

meeting_agent.agent.instructions.role = """
You are a helpful meeting recorder AI agent with access to Google Calendar and Meeting Recording tools.

"""

meeting_agent.agent.instructions.goal = """
Your goal is to help users answer their query, record their meetings, check the status of the recording tools, and get the assets of the recorded meeting using the recording bot id. However, please don't check the status of recorder unless explicitly requested by the user.

"""



#### 5. Usage Examples

In [ ]:
# creates a thread and running the agentic loop
text, thread = meeting_agent.run('Hi! What can you do?')
print(text)

##### Add Calendar Integration:

<div style="border-left: 4px solid #42b983; padding-left: 1em; margin: 1em 0; font-size: 14px; line-height: 1.6">
  <p>
    <strong>🚨 Replace</strong> <code>&lt;DATE&gt;</code> <strong>with the actual date you want to query.</strong><br>
    <span> Format examples:</span> <code>May 21 2025</code>, <code>2025-05-21</code><br>
    <span> Example prompt:</span><br>
    <code>List my upcoming meetings on May 21 2025 and the three consecutive days...</code>
  </p>
</div>


In [ ]:
# adding a message to the existing thread and running the agentic loop
meeting_agent.run('List my upcoming meetings on <DATE> and the three consecutive days, including title, time and participants.', thread_id=thread)

##### Recording Control:

<div style="border-left: 4px solid #42b983; padding-left: 1em; margin: 1em 0; font-size: 14px; line-height: 1.8">
  <p>
    <strong>🚨 Replace</strong> <code>&lt;MEETING_TITLE&gt;</code> <strong>with the exact title of the meeting you want to create a recorder for, from the list above.</strong><br>
    <span> Example prompt:</span><br>
    <code>Create a recorder for the Q2 Planning Sync.</code>
  </p>
</div>


In [ ]:
# adding another message to the existing thread and running the agentic loop
meeting_agent.run('Create a recorder for <MEETING_TITLE>.', thread_id=thread)

In [ ]:
meeting_agent.run('Check the recorder status and give me the asset links if done.', thread_id=thread)

#### Email Delivery:

<div style="border-left: 4px solid #42b983; padding-left: 1em; margin: 1em 0; font-size: 14px; line-height: 1.6">
  <p>
    <strong>🚨 Replace</strong> <code>&lt;YOUR_EMAIL&gt;</code> <strong>with your actual email address.</strong><br>
    <span> Example prompt:</span><br>
    <code>Email the video & transcript to team@example.com with a summary.</code>
  </p>
</div>

In [ ]:
meeting_agent.run('Email the video & transcript to <YOUR_EMAIL>  with a 5‑bullet summary.', thread_id=thread)

#### 6. Next Steps:
#
#### Building on Top of This Agent:

This implementation leverages key capabilities of xpander.ai as a backend for agents, making it ideal for extension:

##### Development Acceleration

- **Visual Workbench**: Rapidly test and debug agent behavior through the xpander.ai Agent Workbench
- **Tool Management**: Add custom tools or use pre-built tools from the library
- **Framework Flexibility**: Switch between different LLM providers without code changes
- **State Persistence**: Thread management across agent calls handled automatically

##### Production Integration

- **Multi-endpoint Triggering**: Expose agents via API, Agent-to-Agent (A2A), MCP, and more.
- **Custom UI Integration**: Wrap agents with a web UI, embed in existing applications, or integrate with Slack/Teams
- **Observability**: Monitor execution paths, performance metrics, and error patterns
- **Versioning & Lifecycle**: Deploy multiple versions and manage agent lifecycle

##### Customization Options

1. **Additional Tools**: Extend the agent with new tools for different meeting platforms or data sources
2. **UI Development**: Build custom interfaces on top of the agent API
3. **Multi-agent Orchestration**: Connect multiple specialized agents working together
4. **Domain Adaptation**: Customize the agent for specific industries or use cases


📚 See the [xpander docs](https://docs.xpander.ai) for deeper integrations.